In [349]:
pip install pycoingecko

Note: you may need to restart the kernel to use updated packages.


In [733]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [734]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from functools import reduce

In [735]:
#import os
#os.getcwd()
#os.chdir("/Users/konstantin/Desktop/dateien/IRTG 1792/CoinGecko/coin_data")

In [736]:
def get_coins_name_id(dataframe_coin):
    coin_ids = dataframe_coin['id']
    coin_names = dataframe_coin['name']
    
    return coin_ids, coin_names

In [737]:
def create_dataframe(coin, coin_id, minimum_date, maximum_date):

    dataframe = []
    timeseries = []
    names = [key for key, val in coin.items()]
    for i in range([len(val) for key, val in coin.items()][0]):
        variables = [item[i][1] for item in coin.values()]
        date = [datetime.fromtimestamp(item[i][0]/1000).strftime("%Y-%m-%d") for item in coin.values()][0]
        dataframe.append(variables)
        timeseries.append(date)
        minimum_date.append(min(timeseries))
        maximum_date.append(max(timeseries))
    
    dataframe = pd.DataFrame(dataframe, columns = names)
    dataframe["Datetime"] = timeseries
    dataframe["Datetime"] = pd.to_datetime(dataframe["Datetime"])
    dataframe["Id"] = coin_id
    dataframe = dataframe.set_index(["Datetime","Id"])
    
    return minimum_date, maximum_date, timeseries, dataframe

In [738]:
def get_df_by_coin_time_range_to_csv(coin_ids, n_start = 0, n_end = 10, 
                              from_time = '1522195200', to_time = '1575158400', Save = False):

    frames = []
    minim_date = []
    maxim_date = []
    id_coin = []
    
    for coin in coin_ids[n_start : n_end]:
        tmp = cg.get_coin_market_chart_range_by_id(id=coin, vs_currency= 'usd', 
                                                   from_timestamp = from_time, to_timestamp = to_time) 
    
        if [val for key, val in tmp.items()][0] == []:
            continue
        else: 
            id_coin.append(coin) 
            
        minim_date, maxim_date, t, dataframe = create_dataframe(tmp, coin, minim_date, maxim_date) 
        
        if Save == True:
            dataframe.to_csv(coin) 
            time.sleep(10)
            frames.append(dataframe)
            
        else:
            frames.append(dataframe)
        
    return frames, minim_date, maxim_date, id_coin 

In [743]:
def main():
    
    coinlist = cg.get_coins_list()
    coinlist_df = pd.DataFrame(coinlist)
    coin_ids, coin_names = get_coins_name_id(coinlist_df)
    
    frames = []
    minim_date = []
    maxim_date = []
    id_coin = []
    
    frames, minim_date, maxim_date, id_coin = get_df_by_coin_time_range_to_csv(coin_ids, n_start = 20, n_end = 22, Save = False)
    
    if minim_date == []:
        print("Coins data available only on minutely/hourly basis. Try with another sequence.")
        
    else:
        index = pd.MultiIndex.from_product([pd.date_range(start= min(minim_date), end=max(maxim_date)).tolist(), 
                                        id_coin], names =['Datetime', 'Id'])  
        match_df = pd.DataFrame(index = index)
        filter_type_frames = match_df.merge(reduce(lambda left,right: pd.concat( [left, right]), frames), how="outer", 
                                    left_index=True, right_index=True).fillna(0)
    
        filter_type_frames.to_csv("test.csv")
        

In [744]:
if __name__== "__main__":
  main()

Coins data available only on minutely/hourly basis. Try with another sequence.


In [ ]:
#visualize coin names
coin_ids, coin_names = get_coins_name_id(coinlist_df)
coin_names